# Домашнее задание 3

1. Для чего и в каких случаях полезны различные варианты усреднения для метрик качества классификации: micro, macro, weighted?

Различные варианты усреднения прежде всего необходимы в задачах, где число классов больше двух (многоклассовая классификация). Они позволяют сравнивать алгоритмы на основе показателей точности, полноты и _F1_, знакомых по задачам бинарной классификации. 
В случае микро-усреднения метрики рассчитываются на основе значений _TP, TN, FP_ и _FN_, оцененных для каждого класса. Таким образом, вклад каждого наблюдения в итоговую оценку одинаковый.
В случае макро-усреднения метрики рассчитываются как арифметическое среднее метрик, определенных для каждого класса в отдельности. Таким образом, данный подход игнорирует возможную несбалансированность данных, так как вклад каждого класса одинаковый.
Взвешенное усреднение также рассчитывается на основе метрик, определенных для каждого класса. Однако в отличие от макро-усреднения веса различаются. Они зависят от размера данного класса в исходных данных. Таким образом, величина класса влияет на итоговую метрику.
Выбор усреднения зависит от конкретной задачи: присутствует ли несбалансированность данных, имеет ли корректная классификация какого-то конкретного класса большее значение.

2. В чём разница между моделями xgboost, lightgbm и catboost или какие их основные особенности?

Модели имеют ряд отличий:
1. Работа с категориальными признаками. XGBoost не умеет работать с такими признаками, поэтому предварительные преобразования должны быть произведены пользователем самостоятельно. LightGBM умеет работать с категориальными признаками, необходимо указать их в соответствующем параметре при обучении модели. Используется собственный алгоритм для преобразования. Следует заметить, что предварительно значения должны быть приведены к типу `int`. CatBoost также умеет работать с категориальными признаками. В данной модели присутствует параметр `one_hot_max_size`, который влияет на то будет применяться для конкретного признака One-Hot Encoding или специальный алгоритм разработанный на основе Mean Encoding. Особенно хорошо CatBoost работает в случаях, когда присутствует много категориальных переменных.
2. В XGBoost для принятия решения о дальнейшем делении дерева используется функция похожести.
3. В модели LightGBM можно выбрать тип используемого бустинга, например GOSS (Gradient-based One-Side Sampling), который быстрее, чем применяемый в XGBoost и CatBoost. В LightGBM также присутствует Exclusive Feature Bundling (EFB), который позволяет совместить несколько признаков в один без потери информации.
4. LightGBM позволяет построение несбалансированных деревьев, что на небольших выборках может приводить к значительному переобучению. Все модели позволяют ограничить глубину деревьев. 
5. CatBoost применяет собственную методику работы с пропущенными значениями, которая гарантирует, что они будут рассматриваться отдельно от остальных. Кроме того данная модель позволяет особенно хорошо бороться с переобучением.

### Дополнительное задание для работы с курсовым проектом

1. Заполните пропуски и выбросы в наиболее проблемных признаках.

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import calendar
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score

In [2]:
df = pd.read_csv('../L1/train_crime.csv')
test = pd.read_csv('../L1/test_crime.csv')
df.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878049 entries, 0 to 878048
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Dates       878049 non-null  object 
 1   Category    878049 non-null  object 
 2   Descript    878049 non-null  object 
 3   DayOfWeek   878049 non-null  object 
 4   PdDistrict  878049 non-null  object 
 5   Resolution  878049 non-null  object 
 6   Address     878049 non-null  object 
 7   X           878049 non-null  float64
 8   Y           878049 non-null  float64
dtypes: float64(2), object(7)
memory usage: 60.3+ MB


In [4]:
df.describe()

,X,Y
count,878049.000000,878049.000000
mean,-122.422616,37.771020
std,0.030354,0.456893
min,-122.513642,37.707879
25%,-122.432952,37.752427
50%,-122.416420,37.775421
75%,-122.406959,37.784369
max,-120.500000,90.000000


Как видим пропущенных значений в датасете нет. С другой стороны была собрана информация о правонарушениях в Сан-Франциско, а это значит, что географические координаты __X__ и __Y__ должны быть в строго заданных границах. Очевидно, что значения широты равные _90_ являются выбросами в заданных условиях.

In [5]:
df[df['Y']==90]

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
660485,2005-12-30 17:00:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Friday,TENDERLOIN,NONE,5THSTNORTH ST / OFARRELL ST,-120.5,90.0
660711,2005-12-30 00:34:00,ASSAULT,INFLICT INJURY ON COHABITEE,Friday,BAYVIEW,"ARREST, BOOKED",JAMESLICKFREEWAY HY / SILVER AV,-120.5,90.0
660712,2005-12-30 00:34:00,ASSAULT,AGGRAVATED ASSAULT WITH BODILY FORCE,Friday,BAYVIEW,"ARREST, BOOKED",JAMESLICKFREEWAY HY / SILVER AV,-120.5,90.0
661106,2005-12-29 00:07:00,NON-CRIMINAL,"AIDED CASE, MENTAL DISTURBED",Thursday,TENDERLOIN,PSYCHOPATHIC CASE,5THSTNORTH ST / EDDY ST,-120.5,90.0
666430,2005-11-30 11:25:00,OTHER OFFENSES,TRAFFIC VIOLATION,Wednesday,TENDERLOIN,"ARREST, CITED",5THSTNORTH ST / ELLIS ST,-120.5,90.0
...,...,...,...,...,...,...,...,...,...
844995,2003-06-11 08:49:00,OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED",Wednesday,INGLESIDE,"ARREST, CITED",JAMES LICK FREEWAY HY / CESAR CHAVEZ ST,-120.5,90.0
845842,2003-06-09 09:25:00,OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED",Monday,INGLESIDE,"ARREST, CITED",JAMES LICK FREEWAY HY / CESAR CHAVEZ ST,-120.5,90.0
852880,2003-05-02 01:00:00,SEX OFFENSES FORCIBLE,"FORCIBLE RAPE, BODILY FORCE",Friday,SOUTHERN,COMPLAINANT REFUSES TO PROSECUTE,3RD ST / JAMES LICK FREEWAY HY,-120.5,90.0
857248,2003-04-14 16:30:00,ROBBERY,"ROBBERY ON THE STREET, STRONGARM",Monday,BAYVIEW,COMPLAINANT REFUSES TO PROSECUTE,GILMAN AV / FITCH ST,-120.5,90.0


In [6]:
df.loc[df['Y']==90, 'X'].value_counts()

-120.5    67
Name: X, dtype: int64

In [7]:
df.loc[df['X']==-120.5, 'X'].value_counts()

-120.5    67
Name: X, dtype: int64

Видим, что максимальные значения для координат __X__ и __Y__ достигаются одновременно. С учетом того, что значение широты выглядит странным для заданной географической границы, это может указывать на то, что данные наблюдения являются выбросами. Создадим дополнительный признак, который будет принимать значение _1_, когда широта равна _90_ или долгота равна _-120.5_. В остальных случаях он будет иметь нулевое значение.

In [8]:
df['GPS_Outlier'] = 0

In [9]:
df.loc[(df['Y'] == 90) | (df['X'] == -120.5), 'GPS_Outlier'] = 1

Преобразуем также даты в более удобный для последующей работы формат.

In [10]:
df['Dates']= pd.to_datetime(df['Dates'], format='%Y-%m-%d %H:%M:%S')

In [11]:
df['Year'] = df['Dates'].dt.year
df['Month_number'] = df['Dates'].dt.month
df['Month_name'] = df['Dates'].dt.month.apply(lambda x: calendar.month_abbr[x])
df['Day'] = df['Dates'].dt.day
df['Hour'] = df['Dates'].dt.hour
df['Minute'] = df['Dates'].dt.minute
df['Second'] = df['Dates'].dt.second
df['DayOfWeek_Number'] = df['Dates'].dt.dayofweek

Теперь признак __Dates__ можно удалить, так как мы извлекли из него всю требуемую информацию. Кроме того мы можем удалить признаки __Descript__ и __Resolution__, так как они присутствуют только в тренировочном наборе данных.

In [12]:
df.drop(columns=['Dates', 'Descript', 'Resolution'], inplace=True)
df.head()

,Category,DayOfWeek,PdDistrict,Address,X,Y,GPS_Outlier,Year,Month_number,Month_name,Day,Hour,Minute,Second,DayOfWeek_Number
0,WARRANTS,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,0,2015,5,May,13,23,53,0,2
1,OTHER OFFENSES,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,0,2015,5,May,13,23,53,0,2
2,OTHER OFFENSES,Wednesday,NORTHERN,VANNESS AV / GREENWICH ST,-122.424363,37.800414,0,2015,5,May,13,23,33,0,2
3,LARCENY/THEFT,Wednesday,NORTHERN,1500 Block of LOMBARD ST,-122.426995,37.800873,0,2015,5,May,13,23,30,0,2
4,LARCENY/THEFT,Wednesday,PARK,100 Block of BRODERICK ST,-122.438738,37.771541,0,2015,5,May,13,23,30,0,2


2. Переведите строковые признаки в числовое представление.

In [13]:
ids = df['Category'].unique()
ids = dict(zip(ids, range(len(ids))))
df['Category_id'] = df['Category'].replace(ids.keys(), ids.values())

In [14]:
y = df['Category_id']
X = df.drop(columns=['Category', 'Category_id'])

In [15]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=0)

Можно заменить строковые признаки, например, частотой с которой они встречаются. Сделаем это для признаков __PdDistrict__ и __Address__.

In [16]:
districts = X_train['PdDistrict'].value_counts().reset_index().rename(columns={'index': 'PdDistrict', 'PdDistrict': 'PdDistrict_freq'})
addresses = X_train['Address'].value_counts().reset_index().rename(columns={'index': 'Address', 'Address': 'Address_freq'})
X_train = X_train.merge(districts, on='PdDistrict', how='left')
X_valid = X_valid.merge(districts, on='PdDistrict', how='left')
X_train = X_train.merge(addresses, on='Address', how='left')
X_valid = X_valid.merge(addresses, on='Address', how='left')
X_valid.head()

,DayOfWeek,PdDistrict,Address,X,Y,GPS_Outlier,Year,Month_number,Month_name,Day,Hour,Minute,Second,DayOfWeek_Number,PdDistrict_freq,Address_freq
0,Friday,BAYVIEW,1000 Block of 23RD ST,-122.390459,37.755187,0,2008,5,May,2,17,30,0,4,66918,27.0
1,Wednesday,MISSION,3600 Block of 18TH ST,-122.425569,37.761403,0,2009,9,Sep,30,18,30,0,2,90091,79.0
2,Tuesday,INGLESIDE,PRECITA AV / SHOTWELL ST,-122.414835,37.747345,0,2007,2,Feb,20,16,33,0,1,59156,8.0
3,Friday,MISSION,18TH ST / CASTRO ST,-122.435003,37.760888,0,2008,11,Nov,14,18,0,0,4,90091,326.0
4,Monday,MISSION,MISSION ST / 16TH ST,-122.419672,37.765050,0,2003,9,Sep,29,18,17,0,0,90091,952.0


3. Обучите любую модель на ваш выбор. Посчитайте метрику, которая подходит для задачи классификации.

In [17]:
Num_features = ['X', 'Y', 'Year', 'Month_Number', 'Day', 'Hour', 
                'Minute', 'Second', 'DayOfWeek_Number', 'PdDistrict_freq', 'Address_freq']

In [18]:
def get_classification_report(y_train_true, y_train_pred, y_test_true, y_test_pred):
    print('TRAIN\n\n' + classification_report(y_train_true, y_train_pred))
    print('TEST\n\n' + classification_report(y_test_true, y_test_pred))
    print('CONFUSION MATRIX\n')
    print(pd.crosstab(y_test_true, y_test_pred))

In [19]:
def evaluate_preds(model, X_train, X_test, y_train, y_test):
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    get_classification_report(y_train, y_train_pred, y_test, y_test_pred)

In [20]:
X_train.drop(columns=['PdDistrict', 'Address', 'DayOfWeek', 'Month_name'], inplace=True)
X_valid.drop(columns=['PdDistrict', 'Address', 'DayOfWeek', 'Month_name'], inplace=True)

In [24]:
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [257]:
%%time
model_catb = OneVsRestClassifier(CatBoostClassifier(silent=True, random_state=21))
model_catb.fit(X_train, y_train)

evaluate_preds(model_catb, X_train, X_valid, y_train, y_valid)

TRAIN

              precision    recall  f1-score   support

           0       0.45      0.08      0.13     31566
           1       0.31      0.51      0.39     94626
           2       0.38      0.77      0.51    131464
           3       0.39      0.37      0.38     40278
           4       0.54      0.07      0.12     33460
           5       0.35      0.22      0.27     69208
           6       0.62      0.08      0.14     17270
           7       0.30      0.22      0.26     57659
           8       0.72      0.11      0.19      6467
           9       0.43      0.11      0.18     27599
          10       0.73      0.04      0.07     23376
          11       0.60      0.12      0.20      3219
          12       0.48      0.20      0.29      7964
          13       0.40      0.51      0.45     40361
          14       0.55      0.05      0.09      3402
          15       0.51      0.05      0.09      7513
          16       0.57      0.12      0.19      5464
          17       0

Результаты оставляют желать лучшего, требуется дальнейшая работа по улучшению модели.

4. Сделайте балансировку данных с помощью пройденных на уроке подходов и еще раз обучите модель.

In [33]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(k_neighbors=3)
X_smote, y_smote = smote.fit_resample(X_train, y_train)

In [34]:
%%time
model_catb_sm = OneVsRestClassifier(CatBoostClassifier(silent=True, random_state=21))
model_catb_sm.fit(X_smote, y_smote)

evaluate_preds(model_catb_sm, X_smote, X_valid, y_smote, y_valid)

KeyboardInterrupt: 